In [1]:
import os
from pinecone import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from langchain_pinecone import PineconeVectorStore

pc = Pinecone(
    api_key=os.getenv("PINECONE_API_KEY"),
    environment="gcp-starter",
)

embeddings = OpenAIEmbeddings()

vector_store = PineconeVectorStore.from_existing_index(
    "recipes",
    embeddings,
)

docs = vector_store.similarity_search("tofu")

docs

/home/gorani/Projects/fullstack-gpt/env/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
